In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [2]:
import logging
import collections
import pathlib

import torch
import numpy as np
import numpy.ma as ma
import pandas as pd

import hyclib as lib
from hyclib.bpd import array as ar

Loaded default config file at /Users/hoyinchau/local_documents/research/utils/utils/config.toml.
Loaded user config files at the following locations from highest to lowest priority: [PosixPath('utils_config.toml'), PosixPath('/Users/hoyinchau/local_documents/research/utils/configs/utils_config.toml'), PosixPath('/Users/hoyinchau/Library/Application Support/utils/config.toml')]


In [3]:
print(lib.cfg['params']['a'])
print(lib.cfg['params']['b'])
print(lib.cfg['params']['c'])
print(lib.cfg['params']['d'])
lib.cfg['params']['b'] = 4
print(lib.cfg['params']['b'])

1st user config
2nd user config
3rd user config
default config
4


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(device)

lib.logging.basic_config()
logger = logging.getLogger()

mps


In [9]:
arr = np.array([0, 1, 2, 10, 11, 20, 21, 22, 23])
chunks = np.array([3, 2, 4])
repeats = np.array([1, 3, 2])
print(lib.np.repeat(arr, repeats, chunks=chunks))

[ 0  1  2 10 11 10 11 10 11 20 21 22 23 20 21 22 23]


In [5]:
data = torch.normal(mean=0.0, std=1.0, size=(10,5))
index = list(range(len(data)))[::-1]
columns = ['a','b','c','d','e']
df = pd.DataFrame(data, index=index, columns=columns)
tdf = lib.bpd.DataFrame(data.to(device), columns=columns)
# tdf[['f','g'], 2:5] = [torch.tensor(1, device=device), np.array(2)]

In [6]:
lib.pd.display(df, max_rows=10)
lib.pd.display(tdf.to_pandas(), max_rows=10)
lib.pd.display(tdf, max_rows=10)

,a,b,c,d,e
9,0.0493,-0.9784,0.8495,0.5414,0.6012
8,1.1839,1.4631,-0.5893,-2.2691,1.2802
7,-0.5478,-0.6529,-1.0229,0.2020,-0.5233
6,-0.7751,-0.7003,0.0834,-1.9348,1.7993
5,1.5634,-0.4210,-0.1522,-1.8284,-0.3722
4,-1.1047,-0.1211,-0.1183,-1.0409,-0.6692
3,2.1641,-1.2725,-0.0892,0.6188,1.3204
2,1.3776,-0.2253,-0.9620,-1.2397,0.2115
1,-2.0523,-0.0980,0.2042,0.0622,0.5168
0,-0.1393,-0.1080,-1.1429,0.1338,-0.4894


,a,b,c,d,e
0,0.0493,-0.9784,0.8495,0.5414,0.6012
1,1.1839,1.4631,-0.5893,-2.2691,1.2802
2,-0.5478,-0.6529,-1.0229,0.2020,-0.5233
3,-0.7751,-0.7003,0.0834,-1.9348,1.7993
4,1.5634,-0.4210,-0.1522,-1.8284,-0.3722
5,-1.1047,-0.1211,-0.1183,-1.0409,-0.6692
6,2.1641,-1.2725,-0.0892,0.6188,1.3204
7,1.3776,-0.2253,-0.9620,-1.2397,0.2115
8,-2.0523,-0.0980,0.2042,0.0622,0.5168
9,-0.1393,-0.1080,-1.1429,0.1338,-0.4894


,atorchfloat32mps:0,btorchfloat32mps:0,ctorchfloat32mps:0,dtorchfloat32mps:0,etorchfloat32mps:0
0,0.0493,-0.9784,0.8495,0.5414,0.6012
1,1.1839,1.4631,-0.5893,-2.2691,1.2802
2,-0.5478,-0.6529,-1.0229,0.2020,-0.5233
3,-0.7751,-0.7003,0.0834,-1.9348,1.7993
4,1.5634,-0.4210,-0.1522,-1.8284,-0.3722
5,-1.1047,-0.1211,-0.1183,-1.0409,-0.6692
6,2.1641,-1.2725,-0.0892,0.6188,1.3204
7,1.3776,-0.2253,-0.9620,-1.2397,0.2115
8,-2.0523,-0.0980,0.2042,0.0622,0.5168
9,-0.1393,-0.1080,-1.1429,0.1338,-0.4894


In [4]:
def rand_string(length, size=None, chars=None, varlength=True):
    if chars is None:
        chars = 'abcdefghijklmnopqrstuvwxyz'
    chars = np.array(list(chars))

    if size is None:
        size = (1,)
    elif isinstance(size, int):
        size = (size,)
    
    if not isinstance(size, tuple):
        raise TypeError(f"size must be a tuple, but {type(size)=}.")
    
    shape = (*size, length)
    s = chars[np.random.randint(len(chars), size=shape)].reshape(-1, length)
    mask = np.random.randint(2, size=s.shape, dtype=bool)
    mask[:,0] = True
    s = np.array([''.join(si[maski]) for si, maski in zip(s, mask)]).reshape(size)
    return s.squeeze()

In [138]:
N, Ma, Mb, chars = 100000, 10, 10, 'pqrstuvwxyz'
data1 = {
    'c': torch.from_numpy(np.random.normal(size=N)),
    'b': rand_string(Mb, size=N, chars=chars),
    'a': np.random.randint(Ma, size=N),
}
data2 = {
    'a': np.random.randint(Ma, size=N),
    'd': rand_string(Mb, size=N, chars=chars),
    'c': np.random.normal(size=N),
}
tdf1 = lib.bpd.DataFrame(data1)
tdf2 = lib.bpd.DataFrame(data2)
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

In [96]:
lib.pd.display(df1)
lib.pd.display(df2)

,c,b,a
0,-0.2727,uvuux,4
1,-0.6567,zxxz,4
2,-0.0333,v,7
...,...,...,...
99997,0.7754,xyzu,9
99998,2.5386,xyy,7
99999,1.7834,wz,9


,a,b,d
0,3,vwxz,0.0949
1,5,vwux,0.2942
2,7,vuw,-0.2334
...,...,...,...
99997,0,wwy,0.1422
99998,6,yzxy,-0.2888
99999,5,zvu,0.3590


In [117]:
%lprun -f lib.bpd.DataFrame.merge -f lib.bpd.concat -f lib.bpd.DataFrame._to_numpy_numeric tdf1.merge(tdf2)

Timer unit: 1e-06 s

Total time: 0.044494 s
File: /Users/hoyinchau/local_documents/research/utils/utils/bpd/dataframe.py
Function: _to_numpy_numeric at line 383

Line #      Hits         Time  Per Hit   % Time  Line Contents
   383                                               def _to_numpy_numeric(self):
   384         1          0.0      0.0      0.0          arrs = []
   385         3          8.0      2.7      0.0          for arr in self._values():
   386         2          9.0      4.5      0.0              arr = arr.detach().cpu().numpy() # do everything in numpy due to various torch.unique bugs
   387         2          3.0      1.5      0.0              if arr.dtype.kind not in 'biufc': # non-numeric
   388         1      44049.0  44049.0     99.0                  _, arr = np.unique(arr, return_inverse=True)
   389         2          3.0      1.5      0.0              if isinstance(arr, ma.MaskedArray):
   390                                                           arr = arr

In [120]:
%%timeit
tdf1.merge(tdf2)

72.1 ms ± 542 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [119]:
%%timeit
df1.merge(df2)

32.6 ms ± 999 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [582]:
(tdf1['a'] == b) & ~(tdf1['b'] == cd)

array([False, False, False, ..., False, False, False])

In [604]:
b = 1
cd = 'x'
expr = '`a` == @b and b == @cd'
lib.pd.display(tdf1.query(expr))
lib.pd.display(df1.query(expr))

,ctorchfloat64cpu,bnumpycpu,anumpyint64cpu
0,-0.7945,x,1
1,0.4098,x,1
2,1.8192,x,1


,c,b,a
35800,-0.7945,x,1
74352,0.4098,x,1
81093,1.8192,x,1


In [139]:
with lib.logging.set_level(logger, 'INFO'):
    tdf12 = tdf1.merge(tdf2, left_on=['a','b'], right_on=['a','d'])
    df12 = df1.merge(df2, left_on=['a','b'], right_on=['a', 'd'])
# out = tdf12.to_pandas().merge(df12, validate='1:1')
# assert len(tdf12) == len(out) == len(df12)
lib.pd.display(tdf12, max_rows=10)
lib.pd.display(df12, max_rows=10)
# lib.pd.display(tdf12.info())
# print(df12.memory_usage())

,c_xtorchfloat64cpu,bnumpycpu,anumpyint64cpu,dnumpycpu,c_ynumpyfloat64cpu
0,-0.6225,pp,0,pp,-0.5483
1,-0.6225,pp,0,pp,-1.4937
2,-0.6225,pp,0,pp,-1.0107
3,0.8804,ppy,0,ppy,0.7557
4,0.0423,zvrq,0,zvrq,1.0064
...,...,...,...,...,...
8753,-0.8050,uy,9,uy,0.1832
8754,-0.8050,uy,9,uy,1.7007
8755,-1.0499,tuyy,9,tuyy,0.2458
8756,-0.6070,zszv,9,zszv,0.3533


,c_x,b,a,d,c_y
0,-0.7422,xrt,2,xrt,2.1341
1,0.6080,zuzt,4,zuzt,0.5829
2,0.0852,zuzt,4,zuzt,0.5829
3,-1.0004,ryyp,6,ryyp,-0.4970
4,-0.6648,ryyp,6,ryyp,-0.4970
...,...,...,...,...,...
8753,0.7385,rtrs,9,rtrs,-1.9875
8754,-0.1353,zusv,2,zusv,-0.5145
8755,1.0079,quty,2,quty,-0.7736
8756,3.0762,zww,8,zww,-0.3875


In [4]:
data1 = {
    'a': [  1,  1,  2,  2,  2,  3,  3],
    'b': ['a','b','c','d','d','e','f'],
    'c': [ 16, 17, 18, 30, 19, 20, 21],
}
data2 = {
    'a': [  1,  1,  2, 2, 13, 14, 15],
    'b': ['e','b','c','d','d','e','f'],
    'd': [ 22, 23, 24, 31, 25, 26, 27],
}

In [8]:
tdf1 = lib.bpd.DataFrame(data1)
tdf2 = lib.bpd.DataFrame(data2)
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
print(df1)
print(df2)
lib.pd.display(tdf1.merge(tdf2), max_rows=np.inf)
lib.pd.display(df1.merge(df2), max_rows=10)

   a  b   c
0  1  a  16
1  1  b  17
2  2  c  18
3  2  d  30
4  2  d  19
5  3  e  20
6  3  f  21
    a  b   d
0   1  e  22
1   1  b  23
2   2  c  24
3   2  d  31
4  13  d  25
5  14  e  26
6  15  f  27


,anumpyint64cpu,bnumpyobjectcpu,cnumpyint64cpu,dnumpyint64cpu
0,1,b,17,23
1,2,c,18,24
2,2,d,30,31
3,2,d,19,31


,a,b,c,d
0,1,b,17,23
1,2,c,18,24
2,2,d,30,31
3,2,d,19,31


In [5]:
class Hi:
    def __init__(self, a):
        self.a = a
        
    def __le__(self, value):
        return self.a <= value.a
    
    def __lt__(self, value):
        return self.a < value.a
    
    def __ge__(self, value):
        return self.a >= value.a
    
    def __gt__(self, value):
        return self.a > value.a
    
    def __eq__(self, value):
        return self.a == value.a

In [26]:
a = np.random.randint(5,size=(30,2))
print(np.unique(a, return_inverse=True, return_counts=True, axis=0))
print(lib.np.unique_rows(a, return_inverse=True, return_counts=True))

(array([[0, 0],
       [1, 0],
       [1, 1],
       [1, 2],
       [1, 3],
       [2, 1],
       [2, 2],
       [2, 3],
       [3, 1],
       [3, 2],
       [4, 1],
       [4, 2],
       [4, 3],
       [4, 4]]), array([13,  8,  7, 12, 11,  3,  5,  1,  4,  8,  6,  1, 13,  9,  8, 13, 11,
        2,  1,  1,  4, 11,  6,  6, 10,  0,  6, 12, 10,  9]), array([1, 4, 1, 1, 2, 1, 4, 1, 3, 2, 2, 3, 2, 3]))
[array([[0, 0],
       [1, 0],
       [1, 1],
       [1, 2],
       [1, 3],
       [2, 1],
       [2, 2],
       [2, 3],
       [3, 1],
       [3, 2],
       [4, 1],
       [4, 2],
       [4, 3],
       [4, 4]]), array([13,  8,  7, 12, 11,  3,  5,  1,  4,  8,  6,  1, 13,  9,  8, 13, 11,
        2,  1,  1,  4, 11,  6,  6, 10,  0,  6, 12, 10,  9]), array([1, 4, 1, 1, 2, 1, 4, 1, 3, 2, 2, 3, 2, 3])]


In [12]:
N, M = 10,3
data = torch.normal(mean=0.0, std=1.0, size=(N,5), requires_grad=True)
index = list(range(len(data)))[::-1]
columns = ['a','b','c','d','e']
df = pd.DataFrame(data.detach().cpu().numpy(), index=index, columns=columns)
tdf = lib.bpd.DataFrame(data.to(device), columns=columns)
tdf['g'] = ma.array(np.random.randint(M, size=N), mask=False)+1
tdf['g', [1,3,5]] = lib.bpd.NA
tdf['g2'] = ma.array(np.random.randint(M, size=N), mask=False)+1
tdf['g2', 2] = lib.bpd.NA
# tdf['g'] = ma.array(1, mask=True)
# tdf['g', [1,3,5]] = [1,2,3]
# tdf['f'] = torch.tensor(2, device=device)
# tdf['f',5:] = 3
tdf['f'] = Hi(2)
tdf['f',5:] = Hi(3)

tdf = tdf.to('cpu')
# tdf['b'] = tdf['b'].numpy()
df = tdf.to_pandas()

In [102]:
arr = tdf[['g','g2','f']].to_numpy()
print(arr.dtype)
print(lib.np.unique_rows(arr))

object


TypeError: Cannot change data-type for object array.

In [82]:
# lib.pd.display(tdf.info(), max_rows=10)
lib.pd.display(tdf, max_rows=10)
# lib.pd.display(tdf.to_pandas(), max_rows=10)

,atorchfloat32cpu,btorchfloat32cpu,ctorchfloat32cpu,dtorchfloat32cpu,etorchfloat32cpu,gnumpyint64cpu,g2numpyint64cpu,fnumpyobjectcpu
0,0.7276,2.0938,-1.2494,-0.0537,-1.6619,2,3,<__main__.Hi object at 0x2b0a41d90>
1,-0.6774,0.3662,0.3469,-0.9446,-0.5408,NA,3,<__main__.Hi object at 0x2b0a41d90>
2,0.7418,0.4353,-1.0761,0.3994,-0.0935,2,NA,<__main__.Hi object at 0x2b0a41d90>
3,-0.4705,-0.7167,0.0153,-1.7831,-1.6039,NA,3,<__main__.Hi object at 0x2b0a41d90>
4,-0.3973,-0.7085,0.8499,0.0501,0.3649,3,1,<__main__.Hi object at 0x2b0a41d90>
5,0.7824,-0.5970,1.6601,0.5834,-0.2487,NA,2,<__main__.Hi object at 0x2b0a41cd0>
6,-0.3310,1.6799,0.8906,-0.7127,1.6212,3,1,<__main__.Hi object at 0x2b0a41cd0>
7,0.9766,-1.7758,-0.6055,-1.4228,-0.0359,2,2,<__main__.Hi object at 0x2b0a41cd0>
8,-0.0070,0.2283,1.6977,0.8107,0.1541,3,1,<__main__.Hi object at 0x2b0a41cd0>
9,-2.2093,-1.2430,-0.2649,0.4869,0.0699,3,3,<__main__.Hi object at 0x2b0a41cd0>


In [13]:
kwargs = {f'{k}_b': ('b', k) for k in ['count', 'sum', 'mean', 'var', 'std', 'sem', 'min', 'max']}
tgdf = tdf.groupby(['g', 'g2', 'f'], dropna=False).agg(
    **kwargs
)
# gdf = df.groupby(['g', 'g2', 'f'], as_index=False, dropna=False).agg(
#     **kwargs
# )

In [109]:
# np.isnan(np.array([None]))

In [14]:
lib.pd.display(tgdf, max_rows=10)
lib.pd.display(gdf, max_rows=10)

,gnumpyint64cpu,g2numpyint64cpu,fnumpyobjectcpu,count_btorchint64cpu,sum_btorchfloat32cpu,mean_btorchfloat32cpu,var_btorchfloat32cpu,std_btorchfloat32cpu,sem_btorchfloat32cpu,min_btorchfloat32cpu,max_btorchfloat32cpu
0,2,1,<__main__.Hi object at 0x2ad2aa190>,1,1.3935,1.3935,nan,nan,nan,1.3935,1.3935
1,2,3,<__main__.Hi object at 0x2ad32e580>,1,1.1087,1.1087,nan,nan,nan,1.1087,1.1087
2,2,3,<__main__.Hi object at 0x2ad2aa190>,2,-0.3420,-0.1710,1.8882,1.3741,0.9717,-1.1426,0.8007
3,3,1,<__main__.Hi object at 0x2ad32e580>,1,-0.4133,-0.4133,nan,nan,nan,-0.4133,-0.4133
4,3,1,<__main__.Hi object at 0x2ad2aa190>,1,-2.2495,-2.2495,nan,nan,nan,-2.2495,-2.2495
5,3,NA,<__main__.Hi object at 0x2ad32e580>,1,0.5733,0.5733,nan,nan,nan,0.5733,0.5733
6,NA,1,<__main__.Hi object at 0x2ad32e580>,1,-1.0455,-1.0455,nan,nan,nan,-1.0455,-1.0455
7,NA,2,<__main__.Hi object at 0x2ad32e580>,1,-1.0498,-1.0498,nan,nan,nan,-1.0498,-1.0498
8,NA,2,<__main__.Hi object at 0x2ad2aa190>,1,-0.2804,-0.2804,nan,nan,nan,-0.2804,-0.2804


NameError: name 'gdf' is not defined

In [38]:
tdf = lib.bpd.DataFrame(data.to(device), columns=columns)
tdf[['f','g'], 2:5] = [torch.tensor(1, device=device), np.array(2)]
print(tdf)
print(tdf.info())
tdf = lib.bpd.DataFrame(data.to(device), columns=columns)
tdf[['f','g'], 2:5] = [torch.tensor([1,2,3], device=device), np.array([2,3,1])]
print(tdf)
print(tdf.info())
tdf = lib.bpd.DataFrame(data.to(device), columns=columns)
tdf[['f','g'], 2:5] = [torch.tensor([1,2,3], device=device)]
print(tdf)
print(tdf.info())
tdf = lib.bpd.DataFrame(data.to(device), columns=columns)
try:
    tdf[['f','g'], 2:5] = torch.tensor([1,2,3], device=device)
except (RuntimeError, ValueError) as err:
    print(err)
tdf = lib.bpd.DataFrame(data.to(device), columns=columns)
try:
    tdf[['f','g'], 2:5] = [1,2,3]
except (RuntimeError, ValueError) as err:
    print(err)

          a         b         c         d         e    f    g
0  1.410362  0.333303  0.405563 -0.932654 -0.497698  NaN  NaN
1 -0.442263 -0.624174 -0.119826 -1.590070 -0.904338  NaN  NaN
2 -0.180317 -0.292788  1.188120  0.383760  0.484597  1.0  2.0
3 -0.173698 -0.172302  0.429156 -1.149883 -0.734977  1.0  2.0
4 -0.024229 -0.546922  0.598683  0.316125 -1.638973  1.0  2.0
5  0.231934  1.055170  0.225477  0.481460 -1.018389  NaN  NaN
6  0.958846 -0.903313 -0.554520 -0.749034 -0.821387  NaN  NaN
7 -0.871738 -1.509601  0.531835 -0.037344 -0.065236  NaN  NaN
8  2.003162 -0.641635  0.637921 -0.516192  1.780537  NaN  NaN
9 -0.657648  0.569463  0.375492  2.407035  0.477876  NaN  NaN
  column  n_rows  is_tensor          dtype device  nbytes
0      a      10       True  torch.float32  mps:0     200
1      b      10       True  torch.float32  mps:0     200
2      c      10       True  torch.float32  mps:0     200
3      d      10       True  torch.float32  mps:0     200
4      e      10       True 

In [50]:
lib.pd.display(tdf.info(), max_rows=10)

,column,n_rows,type,dtype,device,memory
0,a,10,Tensor,torch.float32,mps:0,200
1,b,10,Tensor,torch.float32,mps:0,200
2,c,10,Tensor,torch.float32,mps:0,200
3,d,10,Tensor,torch.float32,mps:0,200
4,e,10,Tensor,torch.float32,mps:0,200
5,f,10,Tensor,torch.int64,mps:0,80
6,g,10,MaskedArray,int64,<NA>,80


In [34]:
tdf[:,['f']] = [torch.tensor(2.0, device=device)]

TypeError: can't convert mps:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [31]:
lib.pd.display(tdf, max_rows=10)
lib.pd.display(tdf.info(), max_rows=10)

,a,b,c,d,e,f
0,0.3389,1.0200,-2.2691,1.2377,-0.8744,2.0000
1,-0.4730,2.7220,1.6786,-0.5806,-1.3611,2.0000
2,-0.5515,1.3892,1.0818,0.0695,-0.9560,2.0000
3,-1.3793,0.0155,-2.1846,0.6579,-0.7420,2.0000
4,1.7557,-0.3933,-1.1636,-0.6708,0.5228,2.0000
5,0.1902,-0.2429,-0.3593,1.3008,-1.7909,2.0000
6,0.9671,0.3975,0.2373,-1.6096,-0.7339,2.0000
7,-0.2327,-0.0719,-1.6113,0.8231,0.3996,2.0000
8,1.5890,1.8805,0.2460,-1.8560,-0.5229,2.0000
9,-1.1374,1.7191,-0.3023,-0.8884,0.8234,2.0000


,column,n_rows,type,dtype,device,memory
0,a,10,Tensor,torch.float32,mps:0,40
1,b,10,Tensor,torch.float32,mps:0,40
2,c,10,Tensor,torch.float32,mps:0,40
3,d,10,Tensor,torch.float32,mps:0,40
4,e,10,Tensor,torch.float32,mps:0,40
5,f,10,Tensor,torch.float32,mps:0,40


In [5]:
data = {'g': [1,2,1,3,4,5,1,6,3,5]}
groups = pd.DataFrame(data, index=df.index)
tgroups = lib.bpd.DataFrame({k: torch.tensor(v) for k, v in data.items()}, index=tdf.index)

data = torch.normal(mean=0.0, std=1.0, size=(2,5))
columns = ['a','b','c','d','e']
rows = pd.DataFrame(data, columns=columns)
trows = lib.bpd.DataFrame(data, columns=columns)

In [6]:
lib.pd.display(pd.concat([df, rows], axis=0))
lib.pd.display(pd.concat([df, groups], axis=1))

,a,b,c,d,e
9,-0.6943,-1.1642,-1.2428,1.4869,-0.9019
8,-1.6453,-0.3682,-0.6364,0.2331,0.5811
7,0.8138,0.5238,0.2277,-0.3891,-0.1911
...,...,...,...,...,...
0,1.1142,-0.3660,-0.5079,-0.0241,0.1670
0,2.1247,-0.0356,0.4185,-0.2265,-1.7851
1,-0.7073,0.4370,1.0251,-1.0811,0.1149


,a,b,c,d,e,g
9,-0.6943,-1.1642,-1.2428,1.4869,-0.9019,1
8,-1.6453,-0.3682,-0.6364,0.2331,0.5811,2
7,0.8138,0.5238,0.2277,-0.3891,-0.1911,1
...,...,...,...,...,...,...
2,-0.2890,-0.6919,-0.5713,0.1333,-0.9726,6
1,0.8237,0.6420,0.4163,0.8803,-0.4724,3
0,1.1142,-0.3660,-0.5079,-0.0241,0.1670,5


In [7]:
lib.pd.display(lib.bpd.concat([tdf, trows], axis=0).reset_index())
lib.pd.display(lib.bpd.concat([tdf, tgroups], axis=1))

,a,b,c,d,e
0,-0.6943,-1.1642,-1.2428,1.4869,-0.9019
1,-1.6453,-0.3682,-0.6364,0.2331,0.5811
2,0.8138,0.5238,0.2277,-0.3891,-0.1911
...,...,...,...,...,...
9,1.1142,-0.3660,-0.5079,-0.0241,0.1670
10,2.1247,-0.0356,0.4185,-0.2265,-1.7851
11,-0.7073,0.4370,1.0251,-1.0811,0.1149


,a,b,c,d,e,g
9,-0.6943,-1.1642,-1.2428,1.4869,-0.9019,1.0000
8,-1.6453,-0.3682,-0.6364,0.2331,0.5811,2.0000
7,0.8138,0.5238,0.2277,-0.3891,-0.1911,1.0000
...,...,...,...,...,...,...
2,-0.2890,-0.6919,-0.5713,0.1333,-0.9726,6.0000
1,0.8237,0.6420,0.4163,0.8803,-0.4724,3.0000
0,1.1142,-0.3660,-0.5079,-0.0241,0.1670,5.0000


In [23]:
lib.pd.display(tdf, max_rows=torch.inf)

,a,b,c,d,e,g
9,-0.8093,0.8262,0.2751,-0.3248,0.9397,1.0000
8,1.2806,-1.4877,-1.0114,-0.8904,0.8248,2.0000
7,-0.7519,-0.1198,-1.5999,0.1825,-0.1673,1.0000
6,0.9651,-0.6341,1.5213,1.0215,-0.0575,3.0000
5,-1.5177,0.6814,0.0637,-0.2516,-1.3175,4.0000
4,-0.4864,0.8057,0.9846,-0.5114,-0.6780,5.0000
3,-1.3489,0.8734,1.5761,-0.8327,0.7568,1.0000
2,-0.3184,0.3598,0.0389,0.6655,0.9025,6.0000
1,-0.2277,0.3524,-0.4454,-0.7042,-0.2016,3.0000
0,-0.2022,-0.0576,-0.9287,1.4529,-0.2721,5.0000


In [124]:
class Hi:
    def __init__(self, a):
        self.a = a
    def __repr__(self):
        return f'Hi({self.a})'
df = pd.DataFrame({'a': [1,2], 'b': torch.tensor([[1,2],[3,4]])})
df[np.array(['d', 'e'])] = [[1,2],[3,4]]
# df[[['d', 'e'],['f','g']]] = [[1,2],[3,4]]
print(df)

ValueError: Data must be 1-dimensional

In [28]:
data = torch.normal(mean=0.0, std=1.0, size=(10,5))
index = list(range(len(data)))[::-1]
columns = ['a','b','c','d','e']
df = pd.DataFrame(data, index=index, columns=columns)
tdf = lib.bpd.DataFrame(data, index=index, columns=columns)

data = {'g': [1,2,1,3,4,5,1,6,3,5]}
groups = pd.DataFrame(data, index=df.index)
tgroups = lib.bpd.DataFrame({k: torch.tensor(v) for k, v in data.items()}, index=tdf.index)

tdf = lib.bpd.concat([tdf, tgroups], axis=1)
tdf.groupby(['g']).agg(
    count=('a', 'count'),
    sum=('a', 'sum'),
    mean=('a', 'mean'),
    var=('a', 'var'),
    std=('a', 'std'),
    sem=('a', 'sem'),
    min=('a', 'min'),
    max=('a', 'max'),
)

     g  count       sum      mean       var       std       sem       min  \
0  1.0    3.0  1.835014  0.611671  0.932795  0.965813  0.557613 -0.417441   
1  2.0    1.0 -0.231530 -0.231530       NaN       NaN       NaN -0.231530   
2  3.0    2.0  1.069503  0.534752  0.506227  0.711497  0.503104  0.031648   
3  4.0    1.0  1.560121  1.560121       NaN       NaN       NaN  1.560121   
4  5.0    2.0  0.188752  0.094376  2.766508  1.663282  1.176118 -1.081742   
5  6.0    1.0  1.990457  1.990457       NaN       NaN       NaN  1.990457   

        max  
0  1.498372  
1 -0.231530  
2  1.037856  
3  1.560121  
4  1.270494  
5  1.990457  

In [53]:
a = pd.DataFrame({
    'a': ['a','c','a','d','b','f','b'],
    'b': [9,8,7,6,5,4,3],
})
print(a.iloc[0])
for k, v in a.groupby('a'):
    print(type(k))
    print(v)

a    a
b    9
Name: 0, dtype: object
<class 'str'>
   a  b
0  a  9
2  a  7
<class 'str'>
   a  b
4  b  5
6  b  3
<class 'str'>
   a  b
1  c  8
<class 'str'>
   a  b
3  d  6
<class 'str'>
   a  b
5  f  4


In [24]:
a = pd.DataFrame({
    'a': [1,2,3,4],
    'b': [4,5,6,7],
})
b = pd.DataFrame({
    'a': [1,3,5],
    'c': [7,8,9]
})
print(a.merge(b, how='right'))

   a    b  c
0  1  4.0  7
1  3  6.0  8
2  5  NaN  9


In [39]:
torch.unique(torch.tensor([
    [1,2,2],
    [1,2,2],
    [1,3,3],
    [3,2,2],
    [1,1,1],
    [3,4,4],
]), return_inverse=True, dim=0)

(tensor([[1, 1, 1],
         [1, 2, 2],
         [1, 3, 3],
         [3, 2, 2],
         [3, 4, 4]]),
 tensor([1, 1, 2, 3, 0, 4]))